# Updating Hosted Feature Layer using Excel Table

#### Ensure that input table (Excel) and target layer (hosted layer) share the same field names
#### The hosted layer is imported from ArcGIS Online.

Reference: https://developers.arcgis.com/python/samples/updating-features-in-a-feature-layer/

In [1]:
from pathlib import Path
import sys

from arcgis.gis import GIS, Item
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from datetime import datetime
import arcpy
import os
import pandas as pd
import numpy as np

In [2]:
from arcgis.features.managers import FeatureLayerManager
from arcgis import features
from arcgis import geometry
from copy import deepcopy

In [93]:
# Define parameters for log in
url_agol = "https://railway-sector.maps.arcgis.com"
username = "matsuzakieijinscrex"
password = "Gorilla@367*"

In [ ]:
# Enter layer item ID
target_layer_itemid = "788a8a39075747739fdd0a446073d695" # for scene layers, choose associated feature layer (do not choose scene layer)

# Enter input table
input_table = r"C:/Users/oc3512/Dropbox/01-Railway/02-NSCR-Ex/01-N2/03-During-Construction/02-Civil/02-Station Structure/01-Masterlist/01-Compiled/N2_Station_Structure_P6ID.xlsx"

In [ ]:
# Define field names:
## Make sure to use correct field names for the followins:

### 1. Join Field
join_field = 'uniqueID'

### 2. OBJECT field names (Esri's default)
objectid_name = 'OBJECTID_1'

### 3. Transfer Field (i.e., field to be updated)
transfer_field = ['target_date']

In [4]:
# Sign in 
gis = GIS(url_agol, username, password)

In [42]:
for i in range(len(featureLayer.layers))[1:2]:
    print(i)

1


# Update Status Field Only

In [90]:
input_table = pd.read_excel(input_table)
input_table.head()

# Import target feature layers from a feature collection
featureLayer = gis.content.get(target_layer_itemid)
for i in range(len(featureLayer.layers))[1:2]: 
    target_layer = featureLayer.layers[i]

    #querying without any conditions returns all the features
    target_fset = target_layer.query()
    target_fset.sdf.head()

    # First get all features
    all_target_features = target_fset.features
    all_target_features[0]

    for_update = []
    uniqueIDs = [f.attributes[join_field] for f in all_target_features] # make sure to obtain uniqueID from feature layer

    for unique_id in uniqueIDs:
        # Status for target
        original_feature = [f for f in all_target_features if f.attributes[join_field] == unique_id][0]
        to_be_updated = deepcopy(original_feature)

        # get the matching row from input_table
        #matching_uniqueid = input_table.where(input_table.uniqueID == unique_id).dropna()

        # get the status for input and target layers
        ## Input 
        id = input_table.index[input_table[join_field] == unique_id]
        status_input = input_table[transfer_field[0]].loc[id].iloc[0]

        ## Target
        status_target = original_feature.attributes[transfer_field[0]]
        objectid_target = original_feature.attributes[objectid_name]

        ## If input and target does not match in status and end_date, append to for_update list
        if status_input != status_target:
            to_be_updated = {
                'attributes': {
                    objectid_name: objectid_target,
                    transfer_field[0] : status_input,
                }
            }
            for_update.append(to_be_updated)

    # Update hosted feature layer
    if len(for_update) == 0:
        print('No need to update target feature layer')
    else:
        print('---------------' + ' Updated Feature Layer ' + '---------------')
        print(target_layer.properties.name + ": ")
        print(for_update)
        target_layer.edit_features(updates=for_update)

--------------- Updated Feature Layer ---------------
Columns: 
[{'attributes': {'OBJECTID_1': 3441, 'Status': 4}}]


# Update Date Field Only

In [94]:
input_table = pd.read_excel(input_table)
input_table.head()

# Import target feature layers from a feature collection
featureLayer = gis.content.get(target_layer_itemid)
for i in range(len(featureLayer.layers))[1:2]: 
    target_layer = featureLayer.layers[i]

    #querying without any conditions returns all the features
    target_fset = target_layer.query()
    target_fset.sdf.head()

    # First get all features
    all_target_features = target_fset.features
    all_target_features[0]

    for_update = []
    uniqueIDs = [f.attributes[join_field] for f in all_target_features] # make sure to obtain uniqueID from feature layer

    for unique_id in uniqueIDs:
        # Status for target
        original_feature = [f for f in all_target_features if f.attributes[join_field] == unique_id][0]
        to_be_updated = deepcopy(original_feature)

        # get dates for input and target layers
        ## Input 
        id = input_table.index[input_table[join_field] == unique_id]
        enddate_input = pd.to_datetime(input_table[transfer_field[0]].loc[id].iloc[0]).date()

        temp = pd.to_datetime(input_table[transfer_field[0]].loc[id])
        temp2 = np.isnat(temp)
        
        # Identify OBJECTID
        objectid_target = original_feature.attributes[objectid_name]
        
        if temp2.bool() == False: # only if enddate_input is not empty, update the date
            to_be_updated = {
                'attributes': {
                    objectid_name: objectid_target,
                    transfer_field[0] : enddate_input,
                }
            }
            for_update.append(to_be_updated)
        else:
            pass
        
    # Update hosted feature layer
    if len(for_update) == 0:
        print('No need to update target feature layer')
    else:
        print('---------------' + ' Updated Feature Layer ' + '---------------')
        print(target_layer.properties.name + ": ")
        #print(for_update)
        target_layer.edit_features(updates=for_update)

--------------- Updated Feature Layer ---------------
Columns: 
